# 🦍 Virunga Intelligence - Notebook d'Analyse ML

Ce notebook est la version interactive du Dashboard Virunga. Il permet d'explorer les données et de tester les algorithmes de Machine Learning (Clustering et NLP) pas à pas.

## 🎯 Objectifs
1.  **Charger les données** (Simulation API Google Sheets).
2.  **Segmenter les activités** (Clustering K-Means) pour identifier les impacts majeurs.
3.  **Analyser la presse** (NLP) pour déterminer le ton médiatique.
4.  **Visualiser les résultats** avec Plotly.

In [ ]:
# --- 1. IMPORTATIONS ---
# Nous importons les librairies nécessaires pour l'analyse de données et le ML.
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# --- CONFIGURATION GRAPHIQUE ---
# Palette de couleurs officielle Virunga
COLORS = {
    "ORANGE": "#EF7D00",
    "FOREST_GREEN": "#00A87D",
    "LAKE_BLUE": "#007D7D",
    "DANGER": "#DC3545",
    "SUCCESS": "#28A745"
}
print("Librairies chargées avec succès.")

## 📥 2. Chargement des Données (Simulation)
Ici, nous simulons les données qui viendraient normalement de Google Sheets via l'API `gspread`.

In [ ]:
def load_mock_data():
    """
    Génère des données de test pour les Activités et la Revue de Presse.
    """
    # --- JEU DE DONNÉES 1 : ACTIVITÉS TERRAIN ---
    activities_data = [
        ["10/31/2025", "jisango@virunga.org", "Cérémonie officielle", "Ville de Goma", "Cérémonie officielle", "La cérémonie officielle journée de l’arbre", "10/24/2025", "1 h - 2 h", "Sud", "Goma", 29, 14, 6],
        ["10/31/2025", "jisango@virunga.org", "Séance éducative", "Secteur Centre", "Sensibilisation", "Sensibilisation Educ Envir & COCO", "10/22/2025", "1 h - 2 h", "Centre", "Nyamilima", 32, 14, 21],
        ["10/31/2025", "bbwende@virunga.org", "Conservation durable", "Kwetu Nature", "Conférence", "Sensibilisation des riverains", "10/31/2025", "2 h - 3 h", "Nord", "Kyavinyonge", 55, 15, 0],
        ["11/3/2025", "bbwende@virunga.org", "Lancement FOB MBAU", "Parc National des Virunga", "Cérémonie officielle", "Lancement des travaux FOB", "10/6/2025", "Plus de 6 h", "Nord", "Mbau", 1500, 10, 0],
        ["11/3/2025", "bbwende@virunga.org", "Remise ration prison", "Parc National des Virunga", "Appui humanitaire", "Aide aux détenus", "10/10/2025", "1 h - 2 h", "Nord", "Beni", 10, 0, 0],
        ["11/22/2025", "jisango@virunga.org", "Séance éducative", "Parc National des Virunga", "Sensibilisation", "Séance avec les femmes", "11/24/2025", "1 h - 2 h", "Centre", "Nyakakoma", 0, 106, 0]
    ]
    df_activities = pd.DataFrame(activities_data, columns=[
        "Timestamp", "Email", "Nom", "Organisateur", "Type", "Description", 
        "Date", "Duree", "Secteur", "Lieu", "Hommes", "Femmes", "Enfants"
    ])
    
    # --- JEU DE DONNÉES 2 : REVUE DE PRESSE ---
    press_data = [
        ["11-Jan-23", "La Fortune RDC", "Lubero : l’ICCN, catalyseur d’une agriculture", "Positif", "Agriculture"],
        ["5-Jan-24", "Opinion Info", "Nord-Kivu : l’équipe du Gouvernorat sacrée", "Positif", "Sport"],
        ["18-Mar-24", "Agora Grands Lacs", "Guerre de l’est : l’ICCN inquiet", "Négatif", "Sécurité"],
        ["23-Oct-25", "Congovert", "Dix naissances de lionceaux", "Très Positif", "Biodiversité"]
    ]
    df_press = pd.DataFrame(press_data, columns=["Date", "Media", "Titre", "Ton", "Thematique"])

    return df_activities, df_press

# Chargement
df_activities, df_press = load_mock_data()
print(f"✅ Données chargées : {len(df_activities)} activités, {len(df_press)} articles de presse.")
display(df_activities.head(3))

## 🧠 3. Intelligence Artificielle : Clustering des Activités

Nous allons utiliser l'algorithme **K-Means** pour grouper automatiquement les activités selon leur ampleur (nombre de participants).
Cela permet de distinguer les *réunions de routine* des *événements majeurs* sans intervention humaine.

In [ ]:
def cluster_activities(df):
    """
    Applique K-Means pour segmenter les activités en 3 clusters basés sur le nombre de participants.
    """
    # 1. Création de la feature 'Total Participants'
    df['Total_Participants'] = df['Hommes'] + df['Femmes'] + df['Enfants']
    
    # 2. Préparation des données pour le modèle (Normalisation)
    features = df[['Total_Participants']].fillna(0)
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)
    
    # 3. Entraînement du modèle K-Means (3 groupes)
    kmeans = KMeans(n_clusters=3, random_state=42)
    df['Cluster'] = kmeans.fit_predict(scaled_features)
    
    # 4. Étiquetage intelligent des clusters
    # On trie les clusters par taille moyenne pour leur donner des noms logiques
    centroids = kmeans.cluster_centers_
    sorted_idx = np.argsort(centroids.flatten())
    cluster_map = {
        sorted_idx[0]: 'Standard (Petit Comité)', 
        sorted_idx[1]: 'Impact Moyen', 
        sorted_idx[2]: 'Impact Majeur (Grand Public)'
    }
    df['Impact_Label'] = df['Cluster'].map(cluster_map)
    
    return df

# Exécution du clustering
df_clustered = cluster_activities(df_activities.copy())

# Affichage des résultats
print("Résultats du Clustering :")
display(df_clustered[['Nom', 'Total_Participants', 'Impact_Label']].sort_values(by='Total_Participants', ascending=False))

### 📊 Visualisation du Clustering
Le graphique ci-dessous montre comment l'IA a séparé les activités. Les couleurs représentent les différents niveaux d'impact.

In [ ]:
fig_cluster = px.scatter(
    df_clustered, 
    x="Hommes", 
    y="Femmes", 
    size="Total_Participants", 
    color="Impact_Label",
    title="Segmentation IA des Activités (K-Means)",
    labels={"Impact_Label": "Niveau d'Impact"},
    color_discrete_map={
        'Standard (Petit Comité)': 'gray', 
        'Impact Moyen': COLORS['LAKE_BLUE'], 
        'Impact Majeur (Grand Public)': COLORS['ORANGE']
    }
)
fig_cluster.update_layout(template="plotly_white")
fig_cluster.show()

## 📰 4. Analyse NLP : Sentiment Analysis

Analyse automatique du ton des articles de presse. Dans une version avancée, nous utiliserions l'API OpenAI (GPT-4) pour une précision maximale.

In [ ]:
def analyze_sentiment_simple(text):
    """
    Analyse basique par mots-clés (Mock pour démo).
    À remplacer par : openai.ChatCompletion.create(...)
    """
    positive_words = ['succès', 'réussite', 'bon', 'paix', 'développement', 'naissance', 'espoir', 'positif']
    negative_words = ['guerre', 'conflit', 'menace', 'décès', 'attaque', 'inquiet', 'négatif']
    
    text_lower = str(text).lower()
    score = 0
    for word in positive_words: 
        if word in text_lower: score += 1
    for word in negative_words: 
        if word in text_lower: score -= 1
        
    if score > 0: return "Positif"
    if score < 0: return "Négatif"
    return "Neutre"

# Application de l'analyse
df_press['Sentiment_IA'] = df_press['Titre'].apply(analyze_sentiment_simple)

# Visualisation Camembert
ton_counts = df_press['Sentiment_IA'].value_counts()
fig_pie = px.pie(
    values=ton_counts.values, 
    names=ton_counts.index, 
    title="Répartition du Ton Médiatique (Analysé par IA)",
    color_discrete_sequence=[COLORS['SUCCESS'], COLORS['DANGER'], COLORS['LAKE_BLUE']]
)
fig_pie.show()

## 🚀 Conclusion

Ce notebook démontre la faisabilité technique du dashboard intelligent.
Le code Python ci-dessus est directement intégré dans l'application `virunga_app.py` pour la mise en production.